<a href="https://colab.research.google.com/github/reic/colab_python/blob/main/Text_to_Epub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Text 轉 epub 程式
import re, os

# check 套件
def chkmodule(libset):
  itm=os.popen("pip3 list").read().splitlines()
  itm=[ data[:data.find(" ")].lower() for data in itm if len(data[:data.find(" ")])>0 ]
  # set(itm)
  if libset.lower() in set(itm):
    return
  os.system(f"pip3 install {libset}")

chkmodule("inlp")
from inlp.convert import chinese

# 基礎設定資料
filename="/content/drive/MyDrive/tmp/\u4FEE\u771F\u804A\u5929\u7FA4.txt" #@param {type:"string"}
title="修真聊天群" #@param {type:"string"}
author="聖騎士的傳說" #@param {type:"string"}
#@markdown 打勾，將會協助進行簡體轉繁體
chinese_S2T = True #@param {type:"boolean"}

# 標題設定義
YAML=f'''---
title: {title}
author: {author}
language: zh-Hant
---'''
# page_break="<div style='page-break-after:always; visibility:hidden'></div>"


with open("title.txt",mode="w",encoding='utf-8') as f:
  f.write(YAML)

with open(filename,mode="r",encoding='utf-8') as f:
  content=f.read()
text=re.sub(r"\n+","\n\n",re.sub(r"[\u3000]+","",content))

print("{:=^50s}".format(" Markdown 處理 "))
# 章/卷處理
patterns=[[r"\n(第.*卷\s?)","#"],[r"\n(第.*章\s?)","#"],[r"\n(第.*節\s?)","#"]]
for pns in patterns:  
  text=re.sub(pns[0],r"\n{} \1".format(pns[1]),text)
print("{}".format("完成..."))
# 建立 md 檔的函數

def writemd(title,arrtomd):
  with open("{:04d}.md".format(title),mode="w",encoding='utf-8') as f:
    f.write(arrtomd)

## 文字處理
textarry=text.split("# ")
counter=0 # counter　為 md 檔名
mdfiles=[] # 記錄 md 檔名
# 若為簡體文件，就需要用 註解的那一個
print("{:=^50s}".format(" md 檔分割 "))

for itm in textarry:
  if chinese_S2T:
    itm=chinese.s2t(itm)
  # writemd(counter,"# {}".format(chinese.s2t(itm)))
  writemd(counter,"# {}".format(itm))
  mdfiles.append("{:04d}.md".format(counter))
  counter+=1

print("{}".format("完成..."))
# counter　為 md 檔名
# mdfiles=["{:04d}.md".format(itm) for itm in range(counter)]

print("{:=^50s}".format(" 產生 epub 檔 "))
# 透過 pandoc 生成 epub 
os.system("pandoc -o \"{}.epub\" title.txt {}".format(title," ".join(mdfiles)))
print("{}".format("完成..."))

# 下載
print("{:=^50s}".format(" 下載 epub 檔 "))
from google.colab import files
files.download('{}.epub'.format(title))